In [1]:
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col


spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()


24/05/04 19:38:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/04 19:38:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
Calls = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/online") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Calls") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

Contacts = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/online") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "Contacts") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [18]:
from pyspark.sql.functions import dense_rank, concat, format_string, lit
from pyspark.sql.window import Window
Calls\
    .join(Contacts, Calls.contact_id == Contacts.id,"inner")\
    .select("contact_id","first_name","type","duration")\
    .withColumn("rnk",dense_rank().over(Window.partitionBy("type").orderBy(col("duration").desc())))\
    .filter(col("rnk")<=3)\
    .withColumn("hours",(col("duration")/3600).cast("int"))\
    .withColumn("minutes",((col("duration")%3600)/60).cast("int"))\
    .withColumn("seconds",(col("duration")%60).cast("int"))\
    .withColumn("time_format",concat(format_string("%02d", col("hours")),lit(":"),
                                     format_string("%02d", col("minutes")),lit(":"),
                                     format_string("%02d", col("seconds"))))\
    .select("first_name","type","time_format")\
    .show()

+----------+--------+-----------+
|first_name|    type|time_format|
+----------+--------+-----------+
|   Michael|incoming|   00:07:00|
|      Jane|incoming|   00:05:00|
|     Emily|incoming|   00:03:00|
|     Alice|outgoing|   00:06:00|
|     Emily|outgoing|   00:04:40|
|      Jane|outgoing|   00:04:00|
+----------+--------+-----------+

